In [16]:
# Connor Murray
# Started 12.14.2024
# Extract metadata from SNPEff annotated VCFs

# Import libraries
import os
import pandas as pd
import gzip
import glob

# Working directory
os.chdir("/standard/vol185/cphg_Manichaikul/users/csm6hg")

# Function to parse the annotation (column 8)
def extract_annotation(info_column):
    """Extract the annotation (ANN=) from the INFO column."""
    for field in info_column.split(";"):
        if field.startswith("ANN="):
            field_value = field.split("=", 1)[1]
            annotation_parts = field_value.split("|")
            if len(annotation_parts) > 4:
                return {
                    "consequence": annotation_parts[1],
                    "gene": annotation_parts[4]
                }
    return {"consequence": None, "gene": None}

# List to store intermediate results
gene_consequence_counts = []

# Read VCF files
vcf_files = glob.glob("topchef_freeze10b/snps_indels50_mac1/*pass_only*.vcf.gz")

# Go through each VCF
for vcf_file in vcf_files:
    print(f"Processing {vcf_file}...")
    
    # Open and read the VCF file in chunks to save memory
    with gzip.open(vcf_file, 'rt') as f:
        # Read the VCF file into a pandas DataFrame, skipping header lines
        vcf_data = pd.read_csv(
            f,
            sep="\t",
            comment="#",
            header=None,
            usecols=range(8), # This makes things so much faster - skip all genotype info
            names=["chrom", "pos", "rsid", "ref", "alt", "qual", "filter", "info"]
        )
    
    # Extract annotation from the INFO column
    annotations = vcf_data["info"].apply(extract_annotation)
    annotation_df = pd.DataFrame(annotations.tolist())  # Convert list of dicts to DataFrame
    
    # Combine extracted annotations with main VCF data
    vcf_data = pd.concat([vcf_data, annotation_df], axis=1)
    
    # Filter out rows with missing annotation
    vcf_data = vcf_data.dropna(subset=["gene", "consequence"])
    
    # Group by gene and consequence, and count the number of SNPs
    grouped = vcf_data.groupby(["gene", "consequence"]).size().reset_index(name="snp_count")
    gene_consequence_counts.append(grouped)

# Combine counts from all files
final_counts = pd.concat(gene_consequence_counts, ignore_index=True)

# Aggregate again in case of overlaps across files
final_counts = final_counts.groupby(["gene", "consequence"]).sum().reset_index()

# Save the final aggregated data
output_file = "gene_consequence_snp_counts.txt.gz"
with gzip.open(output_file, 'wt') as gz_file:
    final_counts.to_csv(gz_file, sep="\t", index=False)

print(f"Aggregated SNP counts saved to {output_file}!")

Processing topchef_freeze10b/snps_indels50_mac1/freeze.10b.chrX.pass_only.snps_indels50_mac1_phased.TOPchef.vcf.gz...
Processing topchef_freeze10b/snps_indels50_mac1/freeze.10b.chr12.pass_only.snps_indels50_mac1_phased.TOPchef.vcf.gz...
Processing topchef_freeze10b/snps_indels50_mac1/freeze.10b.chr11.pass_only.snps_indels50_mac1_phased.TOPchef.vcf.gz...
Processing topchef_freeze10b/snps_indels50_mac1/freeze.10b.chr1.pass_only.snps_indels50_mac1_phased.TOPchef.vcf.gz...
Processing topchef_freeze10b/snps_indels50_mac1/freeze.10b.chr14.pass_only.snps_indels50_mac1_phased.TOPchef.vcf.gz...
Processing topchef_freeze10b/snps_indels50_mac1/freeze.10b.chr16.pass_only.snps_indels50_mac1_phased.TOPchef.vcf.gz...
Processing topchef_freeze10b/snps_indels50_mac1/freeze.10b.chr13.pass_only.snps_indels50_mac1_phased.TOPchef.vcf.gz...
Processing topchef_freeze10b/snps_indels50_mac1/freeze.10b.chr7.pass_only.snps_indels50_mac1_phased.TOPchef.vcf.gz...
Processing topchef_freeze10b/snps_indels50_mac1/fre